O treinamento do modelo se deu sobre um subconjunto dos dados que foi anotado manualmente.  
O conjunto anotado pode ser encontrado em `sample.csv`.

A amostragem foi feita através do script `sample.py`.

In [ ]:
import numpy as np
import pandas as pd
import re

import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
def stratified_sample(df, col, n_per_class):
    return df.groupby(col, group_keys=False).apply(lambda x: x.sample(min(len(x), n_per_class)))

In [ ]:
annotated_data = pd.read_csv('sample.csv')
annotated_data.head()

# Extração de features

A extração de *features* foi feita através de expressões regulares:

Se o título de uma instância corresponder a uma regex em qualquer ponto, associa-se o valor 1 ao atributo da referente à mesma.  
Caso contrário, 0 é atribuído.

In [ ]:
patterns = [
    r'smart', # contém "smart"
    r'(?:ph|f)one', # contém "fone" ou "phone"
    r'\bcelular\b', # contém a palavra "celular"
    r'\bbumper\b', # contém a palavra "bumper"
    r'\b[a-z]\d\b', # contém, p.e., "G5", "S9", ...
    r'\bkit\b', # contém a palavra "kit"
    r'\b(?:capa|case)\b', # contém a palavra "capa" ou "case"
    r'\bpara\b' # contém a palavra "para" (p.e. "antena para celular")
]

patterns_re = [re.compile(pat, re.IGNORECASE) for pat in patterns]

# Usado para nomear as colunas do DataFrame
attr_names = ['smart', 'phone', 'celular', 'bumper', 'letra_num', 'kit', 'capa', 'para']

In [ ]:
# Transforma um título em uma lista de atributos
def get_attributes(title):
    title_attributes = []
    for pattern in patterns_re:
        if pattern.search(title) is None:
            title_attributes.append(0)
        else:
            title_attributes.append(1)
    return title_attributes

## Transformação dos dados

In [ ]:
# Transforma as linhas de um DataFrame nos atributos correspondentes ao título
def get_row_attributes(dataframe):
    attributes = []
    for i in dataframe.index:
        row_attr = get_attributes(dataframe.loc[i].TITLE)
        attributes.append(row_attr)
    attr_df = pd.DataFrame(attributes)
    attr_df.columns = [f're_{re_name}' for re_name in attr_names]
    attr_df.insert(0, 'TITLE', dataframe.TITLE)
    return attr_df

In [ ]:
annotated_attributes = get_row_attributes(annotated_data)
annotated_attributes['SMARTPHONE'] = annotated_data.SMARTPHONE

stratified_sample(annotated_attributes, 'SMARTPHONE', 5)

In [ ]:
sns.heatmap(
    pd.DataFrame(annotated_attributes.iloc[:, 1:].corr().iloc[:-1, -1]),
    vmin=-1,
    vmax=+1,
    cmap=sns.color_palette('RdBu_r', 7))
plt.title('Correlação dos atributos com a variável resposta')
plt.show()

`bumper` e `kit` apresentam uma correlação não existente ou muito pequena com a variável resposta, portanto optei por tirar essas variáveis para reduzir o número de dimensões do problema.

Incluí essas variáveis pois apareciam com certa frequência na amostra anotada, mas parece que não são frequentes ou influentes o bastante para ter uma participação significativa nos resultados.

In [ ]:
patterns = [
    r'smart', # contém "smart"
    r'(?:ph|f)one', # contém "fone" ou "phone"
    r'\bcelular\b', # contém a palavra "celular"
    # r'\bbumper\b', # contém a palavra "bumper"
    r'\b[a-z]\d\b', # contém, p.e., "G5", "S9", ...
    # r'\bkit\b', # contém a palavra "kit"
    r'\b(?:capa|case)\b', # contém a palavra "capa" ou "case"
    r'\bpara\b' # contém a palavra "para" (p.e. "antena para celular")
]

patterns_re = [re.compile(pat, re.IGNORECASE) for pat in patterns]

# Usado para nomear as colunas do DataFrame
attr_names = ['smart', 'phone', 'celular', 'letra_num', 'capa', 'para']

In [ ]:
annotated_attributes = get_row_attributes(annotated_data)
annotated_attributes['SMARTPHONE'] = annotated_data.SMARTPHONE

stratified_sample(annotated_attributes, 'SMARTPHONE', 5)

In [ ]:
sns.heatmap(
    pd.DataFrame(annotated_attributes.iloc[:, 1:].corr().iloc[:-1, -1]),
    vmin=-1,
    vmax=+1,
    cmap=sns.color_palette('RdBu_r', 7))
plt.title('Correlação dos atributos com a variável resposta')
plt.show()

Aqui seria cabível uma análise de componentes principais ou análise de correspondência, mas por questões de tempo e simplicidade, deixei as demais colunas como estão.

# Avaliação de classificadores

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

from scipy.stats import ttest_ind

Foi usada a área sob a curva ROC como medida de avaliação.  
Nesse caso, pode ser interessante ajustar as taxas de falsos e verdadeiros positivos.

Logo a curva ROC é uma representação apropriada para a performance de um classificador, pois sumariza a qualidade dos resultados do classificador sob um *threshold* bom.

Os classificadores testados foram o perceptron e a máquina de vetores de suporte com kernel polinomial de grau 3.  
Os resultados foram comparados com um teste T pareado para amostras independentes.

In [ ]:
X = annotated_attributes.iloc[:, 1:-1].values
y = annotated_attributes.iloc[:, -1].values

In [ ]:
def eval_clf(clf):
    roc_auc = cross_val_score(
        clf,
        X,
        y,
        scoring='roc_auc',
        cv=10
    )
    print(f'roc_auc: {np.mean(roc_auc):.2f} +- {np.std(roc_auc):.2f}\t'
         + f'min: {np.min(roc_auc):.2f}; max: {np.max(roc_auc):.2f}')
    return roc_auc

In [ ]:
perceptron = Perceptron(max_iter=1e3)

print('Perceptron:')
roc_auc_per = eval_clf(perceptron)

In [ ]:
svc = SVC(gamma='auto', kernel='poly', degree=3) # sem limite de iterações

print('SVC (poly-3):')
roc_auc_svc = eval_clf(svc)

In [ ]:
pvalue = ttest_ind(roc_auc_per, roc_auc_svc).pvalue
print(f'p-value: {pvalue:.3f} (5% de significância)')
if pvalue < 0.05:
    print('Há evidências de que os classificadores possuem performances diferentes')
else:
    print('Não há evidência de diferença de performance')

## Seleção do classificador

Não houve evidências para dizer que um dos classificadores teve performance melhor.  
Logo, optei pela simplicidade do perceptron em detrimento do SVC.  
Entre os benefícios de optar pelo modelo simples, há a chance reduzida de overfitting.

# Transformação do dataset

## Treino do classificador escolhido

In [ ]:
classifier = perceptron

X = annotated_attributes.iloc[:, 1:-1].values
y = annotated_attributes.iloc[:, -1].values
classifier.fit(X, y)

In [ ]:
data = pd.read_csv('data_estag_ds.tsv', sep='\t')
data.head()

In [ ]:
data_attr = get_row_attributes(data)
data_attr.head()

In [ ]:
predictions = classifier.predict(data_attr.iloc[:, 1:].values)

In [ ]:
answers = pd.DataFrame()
answers['ID'] = data.ID
answers['TITLE'] = data.TITLE
answers['SMARTPHONE'] = pd.Series(predictions)

In [ ]:
stratified_sample(answers, 'SMARTPHONE', 5)

In [ ]:
answers.to_csv('output.tsv', sep='\t', index=False)